# 作業 : (Kaggle)鐵達尼生存預測

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察計數編碼與特徵雜湊的效果

# [作業重點]
- 仿造範例, 完成自己挑選特徵的群聚編碼 (In[2], Out[2])
- 觀察群聚編碼, 搭配邏輯斯回歸, 看看有什麼影響 (In[5], Out[5], In[6], Out[6]) 

# 作業1
* 試著使用鐵達尼號的例子，創立兩種以上的群聚編碼特徵( mean、median、mode、max、min、count 均可 )

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression, LinearRegression
import warnings
warnings.filterwarnings('ignore')

data_path = 'Part02/'
df = pd.read_csv(data_path + 'titanic_train.csv')

train_Y = df['Survived']
df = df.drop(['PassengerId', 'Survived'] , axis=1)
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# 取一個類別型欄位, 與一個數值型欄位, 做群聚編碼
df['Sex'] = df['Sex'].fillna('None')
mean_df = df.groupby(['Sex'])['Age'].mean().reset_index()
mode_df = df.groupby(['Sex'])['Age'].apply(lambda x: x.mode()).reset_index()
median_df = df.groupby(['Sex'])['Age'].median().reset_index()
max_df = df.groupby(['Sex'])['Age'].max().reset_index()
mode_df
#temp = pd.merge(mean_df, mode_df,how='left',on=['Sex'])
#temp = pd.merge(temp, median_df,how='left',on=['Sex'])
#temp = pd.merge(temp, max_df,how='left',on=['Sex'])
#temp.columns = ['Sex','Cabin_age_Mean', 'Cabin_age_Mode', 'Cabin_age_Median', 'Cabin_age_Max']
#temp

,Sex,level_1,Age
0,female,0,24.0
1,male,0,19.0
2,male,1,25.0
3,male,2,28.0


In [3]:
df = pd.merge(df, temp, how='right',on=['Sex'])
df = df.drop(['Sex'] , axis=1)
df.head()

,Pclass,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_age_Mean,Cabin_age_Mode,Cabin_age_Median,Cabin_age_Max
0,3,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,30.726645,19.0,29.0,80.0
1,3,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,30.726645,19.0,29.0,80.0
2,3,"Moran, Mr. James",NaN,0,0,330877,8.4583,NaN,Q,30.726645,19.0,29.0,80.0
3,1,"McCarthy, Mr. Timothy J",54.0,0,0,17463,51.8625,E46,S,30.726645,19.0,29.0,80.0
4,3,"Palsson, Master. Gosta Leonard",2.0,3,1,349909,21.0750,NaN,S,30.726645,19.0,29.0,80.0


In [21]:
df['Embraked'].shape[0]

998

In [4]:
#只取 int64, float64 兩種數值型欄位, 存於 num_features 中
num_features = []
for dtype, feature in zip(df.dtypes, df.columns):
    if dtype == 'float64' or dtype == 'int64':
        num_features.append(feature)
print(f'{len(num_features)} Numeric Features : {num_features}\n')

# 削減文字型欄位, 只剩數值型欄位
df = df[num_features]
df = df.fillna(-1)
MMEncoder = MinMaxScaler()
df.head()

9 Numeric Features : ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'Cabin_age_Mean', 'Cabin_age_Mode', 'Cabin_age_Median', 'Cabin_age_Max']



,Pclass,Age,SibSp,Parch,Fare,Cabin_age_Mean,Cabin_age_Mode,Cabin_age_Median,Cabin_age_Max
0,3,22.0,1,0,7.2500,30.726645,19.0,29.0,80.0
1,3,35.0,0,0,8.0500,30.726645,19.0,29.0,80.0
2,3,-1.0,0,0,8.4583,30.726645,19.0,29.0,80.0
3,1,54.0,0,0,51.8625,30.726645,19.0,29.0,80.0
4,3,2.0,3,1,21.0750,30.726645,19.0,29.0,80.0


# 作業2
* 將上述的新特徵，合併原有的欄位做生存率預估，結果是否有改善?

In [5]:
# 原始特徵 + 邏輯斯迴歸
# 沒有這四個新特徵的 dataframe 稱為 df_minus
df_original = df.drop(['Cabin_age_Mean','Cabin_age_Mode','Cabin_age_Median','Cabin_age_Max'] ,axis=1)
train_X = MMEncoder.fit_transform(df_original)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6150413134041685

In [6]:
# 新特徵 + 邏輯斯迴歸
"""
Your Code Here
"""
train_X = MMEncoder.fit_transform(df)
estimator = LogisticRegression()
cross_val_score(estimator, train_X, train_Y, cv=5).mean()

0.6150413134041685

In [23]:
df = pd.DataFrame({'A': [1, 2, 1, 2, 1, 2, 3]})

KeyError: 0

In [55]:
#coding=utf-8
import pandas as pd 
import numpy as np 

left = pd.DataFrame({
    'key':['K0','K1','K2','K3'],
    'A':['A0','A1','A2','A3'],
    'B':['B0','B1','B2','B3']})

right = pd.DataFrame({
    'key':['K0','K1','K2','K3'],
    'C':['C0','C1','C2','C3'],
    'D':['D0','D1','D2','D3']})

print(left)
'''
    A   B key
0  A0  B0  K0
1  A1  B1  K1
2  A2  B2  K2
3  A3  B3  K3
'''
print(right)
'''
    C   D key
0  C0  D0  K0
1  C1  D1  K1
2  C2  D2  K2
3  C3  D3  K3
'''

# 目標，基於 key 把 left 與 right 合併
# 使用 merge
res = pd.merge(left,right,how='outer', on='key')
print(res)
'''
    A   B key   C   D
0  A0  B0  K0  C0  D0
1  A1  B1  K1  C1  D1
2  A2  B2  K2  C2  D2
3  A3  B3  K3  C3  D3
'''

  key   A   B
0  K0  A0  B0
1  K1  A1  B1
2  K2  A2  B2
3  K3  A3  B3
  key   C   D
0  K0  C0  D0
1  K1  C1  D1
2  K2  C2  D2
3  K3  C3  D3
  key   A   B   C   D
0  K0  A0  B0  C0  D0
1  K1  A1  B1  C1  D1
2  K2  A2  B2  C2  D2
3  K3  A3  B3  C3  D3


'\n    A   B key   C   D\n0  A0  B0  K0  C0  D0\n1  A1  B1  K1  C1  D1\n2  A2  B2  K2  C2  D2\n3  A3  B3  K3  C3  D3\n'